### Install dependencies

In [1]:
from jupyter_dash import JupyterDash
import pandas as pd
import numpy as np
from plotly import graph_objects as go
import app as app_file
import setup as setup_file
import dash_core_components as dcc
import dash_bootstrap_components as dbc
import pickle
from configuration import Config
from statistics import Statistics

### Setup the environment

In [ ]:
JupyterDash.infer_jupyter_proxy_config()

### Configure a genetic algorithm

In [4]:
configs = Config()
setup_applet = setup_file.create_app(configs)
setup_applet.run_server(mode='inline')

### Create a sample population

In [5]:
sample_pop = configs.params['enc_type'].initialize()
print(sample_pop[0])

{'gene': array([11, 29, 59, 17, 12, 26,  6, 21, 14, 43, 50, 16,  5, 40, 25, 20, 32,
       28, 23, 10, 57, 27, 18, 24, 33,  2, 63, 58, 52, 49, 31, 44, 45, 55,
       34, 56, 38, 39, 30, 54, 41, 35, 13, 61, 62, 36, 15, 47, 60, 46,  9,
       19, 42,  1, 48,  7,  8, 37,  0, 22,  3, 51,  4, 53]), 'fitness': array([0])}


### Define a custom evaluation function

Your evaluation function should update all fitness values of an individual. Sample evaluation functions for the predefined problems can be found in ```evaluate.py```



In [6]:
def custom_eval(self, pop):
    return pop

configs.params['eval_type'] = cust_eval

In [6]:
sample_pop = configs.params['eval_type'](configs, sample_pop)
print(sample_pop[0])

{'gene': array([7, 3, 6, 1, 5, 0, 4, 2]), 'fitness': array([4]), 'meta': {}}


### Define a custom visualization

Your visualization function should define a phenotypic representation of an individual using the ```dash_core_components``` or ```dash_bootstrap_components``` libraries (see ```README.md```). Sample visualization functions for the predefined problems can be found in ```visualize.py```. If you don't want to define a custom visualization, don't run the following cell.


In [7]:
def cust_vis(ind):
    return dcc.Graph(fig=go.Figure())

configs.params['cust_vis'] = cust_vis

### Run the genetic algorithm

In [5]:
stats = Statistics(configs.params)
main_app = app_file.create_app(configs, stats)
main_app.run_server(mode='inline')

### Analyze the resulting population

In [14]:
population = stats.posthoc['population']
population

{'all': [array([78, 60, 25, 12, 89,  6, 84, 23,  7, 51, 70, 34, 76, 67, 63,  5, 68,
         11, 19, 22, 17, 71,  8, 26, 40, 69, 72, 85, 33, 44, 45, 98, 42, 31,
         24, 43, 29, 37, 46,  0, 13,  2, 87, 48, 95, 80, 81, 74, 49, 66, 16,
         53, 21, 79, 82,  4, 47, 20, 35, 28, 94, 39, 90, 75, 61, 57, 73, 36,
         62, 38,  9, 55, 83, 14, 18,  1, 96, 52, 30, 77, 91, 56, 59,  3, 27,
         92, 50, 88, 64, 65, 32, 97, 54, 10, 86, 58, 15, 99, 41, 93]),
  array([78, 60, 25, 12, 89,  6, 84, 23,  7, 51, 70, 34, 76, 67, 63,  5, 68,
         11, 19, 22, 17, 71,  8, 26, 40, 69, 72, 85, 33, 44, 45, 98, 42, 31,
         24, 43, 29, 37, 46,  0, 13,  2, 87, 48, 95, 80, 81, 74, 49, 66, 16,
         53, 21, 79, 82,  4, 47, 20, 35, 28, 94, 39, 90, 75, 61, 57, 73, 36,
         62, 38,  9, 55, 83, 14, 18,  1, 96, 52, 30, 77, 91, 56, 59,  3, 27,
         92, 50, 88, 64, 65, 32, 97, 54, 10, 86, 58, 15, 99, 41, 93]),
  array([78, 60, 25, 12, 89,  6, 84, 23,  7, 51, 70, 34, 76, 67, 63,  5, 68,
    